In [1]:
import sys
import os
from src.model import model
from src.model_outerweights import model_outerweights
from src.greedy_insertion import insertion
import numpy as np
from loguru import logger
import torch

Load the data that is generated from the open-loop optimization

In [2]:
# load the data
path = '../data_result/raw_data/VDP_beta_0.1_grid_combined.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-08-04 09:23:21.149 | INFO     | __main__:<module>:4 - Loaded data with shape: (1800,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


set up the testing parameter.

In [20]:
n_neurons = 100
weights = np.random.randn(n_neurons, 2) * 0.1
bias = np.random.randn(n_neurons) * 0.1
outer_weights = np.random.randn(n_neurons) * 0.1
regularization = (5, 0.01)

The semi-smooth newton method works in general fine 

In [18]:
# Using the new class interface
vdp_model = model(data, torch.relu, 2.0, regularization, optimizer='SSN', loss_weights=(1.0, 1.0))
model_result, weight, bias, output_weight = vdp_model.train(
    inner_weights=weights, inner_bias=bias, 
    iterations=10000,
    display_every=1000
)

2025-08-04 09:33:03 | INFO     | src.model:_configure_logger:82 - VDPModel initialized
2025-08-04 09:33:03 | INFO     | src.model:train:245 - Starting network training session
2025-08-04 09:33:03 | INFO     | src.model:_prepare_data:126 - Training set: 1620 samples, Validation set: 180 samples
2025-08-04 09:33:03 | INFO     | src.model:_prepare_data:129 - Data ranges - x: [-3.00, 3.00], v: [0.00, 10.96], dv: [-13.19, 13.19]
2025-08-04 09:33:03 | INFO     | src.model:_create_network:162 - Creating network with 100 neurons
2025-08-04 09:33:03 | INFO     | src.model:_setup_optimizer:176 - Using SSN optimizer with alpha=0.01, gamma=5
2025-08-04 09:33:03 | INFO     | src.model:train:263 - Training model, saving to /Users/ruizhechao/Documents/NNforHJB/train_history
2025-08-04 09:33:03 | INFO     | src.model:train:265 - Training hyperparameters: iterations=10000, batch_size=1620, display_every=1000
2025-08-04 09:33:03 | INFO     | src.model:train:266 - Loss weights: value=1.0, gradient=1.0
20

In [21]:
vdp_model = model_outerweights(data, torch.relu, 2.0, regularization, optimizer='SSN', loss_weights=(1.0, 0.0))
model_result, weight, bias, output_weight = vdp_model.train(
    inner_weights=weights, inner_bias=bias, outer_weights=outer_weights,
    iterations=10000,
    display_every=500
)

2025-08-04 09:36:49 | INFO     | src.model_outerweights:_configure_logger:82 - VDPModel (outer weights) initialized
2025-08-04 09:36:49 | INFO     | src.model_outerweights:train:245 - Starting network training session (outer weights only)
2025-08-04 09:36:49 | INFO     | src.model_outerweights:_prepare_data:126 - Training set: 1620 samples, Validation set: 180 samples
2025-08-04 09:36:49 | INFO     | src.model_outerweights:_prepare_data:129 - Data ranges - x: [-3.00, 3.00], v: [0.00, 10.96], dv: [-13.19, 13.19]
2025-08-04 09:36:49 | INFO     | src.model_outerweights:_setup_optimizer:169 - Using SSN optimizer with alpha=0.01, gamma=5
2025-08-04 09:36:49 | INFO     | src.model_outerweights:train:261 - Training model, saving to /Users/ruizhechao/Documents/NNforHJB/train_history
2025-08-04 09:36:49 | INFO     | src.model_outerweights:train:263 - Training hyperparameters: iterations=10000, batch_size=1620, display_every=500
2025-08-04 09:36:49 | INFO     | src.model_outerweights:train:264 -